## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-14-04-01-23 +0000,nyt,It’s Paradise Lost as Climate Change Remakes E...,https://www.nytimes.com/2025/07/14/world/europ...
1,2025-07-14-04-01-15 +0000,nyt,Scotland’s Unreliable Ferries Leave People on ...,https://www.nytimes.com/2025/07/14/world/europ...
2,2025-07-14-03-50-12 +0000,nypost,JD Vance claps back at snide comment from Gov....,https://nypost.com/2025/07/13/us-news/jd-vance...
3,2025-07-14-03-26-00 +0000,wsj,China’s Exports Beat Expectations in Boost for...,https://www.wsj.com/economy/trade/chinas-expor...
4,2025-07-14-02-51-00 +0000,wsj,U.S. Policy Shifts May Dominate Dollar’s Tone ...,https://www.wsj.com/finance/currencies/asian-c...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2374/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
81,trump,9
127,nyc,5
177,mamdani,5
206,president,4
105,he,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
7,2025-07-14-02-00-00 +0000,wsj,Chinese manufacturers looking to dodge steep U...,https://www.wsj.com/economy/trade/vietnam-chin...,31
9,2025-07-14-01-05-35 +0000,nypost,Trump announces US will send Patriot missiles ...,https://nypost.com/2025/07/13/world-news/trump...,29
16,2025-07-13-22-43-30 +0000,nypost,Ex-AOC top aide urges Mamdani to run primaries...,https://nypost.com/2025/07/13/us-news/ex-aoc-a...,27
31,2025-07-13-19-11-43 +0000,nypost,President Trump and first lady Melania arrive ...,https://nypost.com/2025/07/13/us-news/presiden...,26
15,2025-07-13-22-44-12 +0000,nypost,Ex-Secret Service director fires back at Rand ...,https://nypost.com/2025/07/13/us-news/ex-secre...,26


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
7,31,2025-07-14-02-00-00 +0000,wsj,Chinese manufacturers looking to dodge steep U...,https://www.wsj.com/economy/trade/vietnam-chin...
16,27,2025-07-13-22-43-30 +0000,nypost,Ex-AOC top aide urges Mamdani to run primaries...,https://nypost.com/2025/07/13/us-news/ex-aoc-a...
19,20,2025-07-13-21-33-00 +0000,nypost,Eric Adams rips Andrew Cuomo’s planned bid to ...,https://nypost.com/2025/07/13/us-news/adams-ch...
9,20,2025-07-14-01-05-35 +0000,nypost,Trump announces US will send Patriot missiles ...,https://nypost.com/2025/07/13/world-news/trump...
11,19,2025-07-13-23-10-41 +0000,nypost,Devoted dad gunned down en route to pick up da...,https://nypost.com/2025/07/13/us-news/nyc-dad-...
31,17,2025-07-13-19-11-43 +0000,nypost,President Trump and first lady Melania arrive ...,https://nypost.com/2025/07/13/us-news/presiden...
13,15,2025-07-13-22-52-32 +0000,nypost,Nassau County will allow cops to wear face mas...,https://nypost.com/2025/07/13/us-news/nassau-c...
15,15,2025-07-13-22-44-12 +0000,nypost,Ex-Secret Service director fires back at Rand ...,https://nypost.com/2025/07/13/us-news/ex-secre...
29,14,2025-07-13-19-31-09 +0000,nypost,Camp Mystic survivors heard screams in the dar...,https://nypost.com/2025/07/13/us-news/camp-mys...
17,14,2025-07-13-22-42-34 +0000,nypost,‘Chic’ California enclave sends homeless packi...,https://nypost.com/2025/07/13/us-news/chic-cal...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
